In [ ]:

!pip install -q accelerate transformers peft bitsandbytes pillow
from huggingface_hub import login
HF_TOKEN = input("Enter your Hugging Face token: ").strip()
login(HF_TOKEN)
from transformers import AutoProcessor, AutoModelForImageTextToText
from peft import PeftModel
import torch
from PIL import Image
from google.colab import files

model_id = "google/medgemma-4b-it"
adapter_id = "rakesh2003/xraymedgemmalora"

print("🔄 Loading base model...")
base_model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
).to("cuda")

print("🔄 Loading YOUR finetuned adapters from HuggingFace...")
model = PeftModel.from_pretrained(base_model, adapter_id)
model = model.to("cuda")

processor = AutoProcessor.from_pretrained(model_id)
print("✅ Your model loaded successfully from HuggingFace!")


PROMPT_TEMPLATE = """Analyze this chest X-ray and provide:
1) Abnormalities
2) Diagnosis
3) Severity
4) Recommendations
Be concise."""

print("\n📤 Upload a chest X-ray image...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
sample_img = Image.open(filename).convert("RGB").resize((128, 128))

messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are an expert radiologist."}]},
    {"role": "user", "content": [
        {"type": "text", "text": PROMPT_TEMPLATE},
        {"type": "image", "image": sample_img}
    ]}
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to("cuda", dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

print("🔮 Generating diagnosis...")
with torch.inference_mode():
    generation = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False
    )[0][input_len:]

result = processor.decode(generation, skip_special_tokens=True)

print("\n" + "="*70)
print("🎯 DIAGNOSIS")
print("="*70)
print(result)
print("="*70)

from IPython.display import display
display(Image.open(filename).resize((256, 256)))
